# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing functions of pandas and sklearn.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use DataFrames to do some feature engineering
* Use built-in sklearn functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Import pandas

In [1]:
import pandas as pd

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

# Load training and testing data

In [3]:
train_data = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

In [4]:
from sklearn.linear_model import LinearRegression

example_features = ['sqft_living', 'bedrooms', 'bathrooms']
X = train_data[example_features]
y = train_data['price']

reg = LinearRegression()
example_model = reg.fit(X, y)

Now that we have fitted the model we can extract the regression weights (coefficients) as follows:

In [5]:
example_weight_summary = example_model.coef_
print example_weight_summary

[   315.40669062 -65081.88711588   6942.16598637]


# Making Predictions

In [6]:
example_predictions = example_model.predict(X)
print example_predictions[0]

271789.26538


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [7]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)

    # Then compute the residuals/errors
    residuals = predictions - outcome

    # Then square and add them up
    residuals_squared = residuals * residuals
    RSS = residuals_squared.sum()

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [8]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data[example_features], test_data['price'])
print rss_example_train

2.73761940583e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [9]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [10]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [11]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']

test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [12]:
for feature in ['bedrooms_squared', 'bed_bath_rooms', 'log_sqft_living', 'lat_plus_long']:
    print 'mean of {0} is {1}'.format(feature, test_data[feature].mean())

mean of bedrooms_squared is 12.4466777016
mean of bed_bath_rooms is 7.50390163159
mean of log_sqft_living is 7.55027467965
mean of lat_plus_long is -74.653333554


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [13]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' and look at the value of the weights/coefficients:

In [14]:
# Learn the three models:
X1 = train_data[model_1_features]
X2 = train_data[model_2_features]
X3 = train_data[model_3_features]

model1 = LinearRegression().fit(X1, y)
model2 = LinearRegression().fit(X2, y)
model3 = LinearRegression().fit(X3, y)

In [15]:
# Examine/extract each model's coefficients:
print 'coefficients of model1: {0}'.format(model1.coef_)
print 'coefficients of model2: {0}'.format(model2.coef_)
print 'coefficients of model3: {0}'.format(model3.coef_)

coefficients of model1: [  3.12258646e+02  -5.95865332e+04   1.57067421e+04   6.58619264e+05
  -3.09374351e+05]
coefficients of model2: [  3.06610053e+02  -1.13446368e+05  -7.14613083e+04   6.54844630e+05
  -2.94298969e+05   2.55796520e+04]
coefficients of model3: [  5.29422820e+02   3.45142296e+04   6.70607813e+04   5.34085611e+05
  -4.06750711e+05  -8.57050439e+03  -6.78858667e+03  -5.61831484e+05
   1.27334900e+05]


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [16]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
print 'RSS on TRAINING data for model1: {0}'.format(get_residual_sum_of_squares(model1, X1, y))
print 'RSS on TRAINING data for model2: {0}'.format(get_residual_sum_of_squares(model2, X2, y))
print 'RSS on TRAINING data for model3: {0}'.format(get_residual_sum_of_squares(model3, X3, y))

RSS on TRAINING data for model1: 9.6787996305e+14
RSS on TRAINING data for model2: 9.58419635074e+14
RSS on TRAINING data for model3: 9.0343645505e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [17]:
# Compute the RSS on TESTING data for each of the three models and record the values:
print 'RSS on TESTING data for model1: {0}'.format(get_residual_sum_of_squares(model1, test_data[model_1_features], test_data['price']))
print 'RSS on TESTING data for model2: {0}'.format(get_residual_sum_of_squares(model2, test_data[model_2_features], test_data['price']))
print 'RSS on TESTING data for model3: {0}'.format(get_residual_sum_of_squares(model3, test_data[model_3_features], test_data['price']))

RSS on TESTING data for model1: 2.25500469795e+14
RSS on TESTING data for model2: 2.23377462976e+14
RSS on TESTING data for model3: 2.59236319207e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.